In [73]:
username = "Alex"
pathtorepo = "C:\\Users\\" *username *  "\\Desktop\\"
using Pkg
Pkg.activate(pathtorepo * "dynamical-systems\\env\\integrate\\")
include(pathtorepo * "dynamical-systems\\system.jl")

  Activating project at `C:\Users\Alex\Desktop\dynamical-systems\env\integrate`

rate_jac (generic function with 1 method)

In [74]:
using StaticArrays, DifferentialEquations, DynamicalSystems
sE, sI, rE, rI, Y  = -50..50, -50..50,  -50..50, -50..50, -50..50
box = sE × sI × rE × rI × Y
using GLMakie, LinearAlgebra

In [75]:
# (+1.92522693, +1.92525595)

In [76]:
τsE = 3.0; γE = 4.0; s0E = 0.15;
τsI = 10.0; γI = 8.0; s0I = 0.6441371581267546
# IE = 0.9; 
τrE = 2.0; kE = 5.0; IE = 1.9251587331649411; wEE = 3.5; wIE = 5.0; θE = 0.2;
τrI = 6.0; kI = 5.0; II = 0.0; wEI = 5.0; wII = 3.0; θI = 0.4;

τY = 0.01;  βY = 0.0;
ythr = 0.5; sEthr = 0.5; kY = 0.01; γY = 0.0;

In [77]:
time = 1000; tt = 0.0; tstep = 0.001; times = [time, tt]
integ_set = (alg = Vern9(), adaptive = false, dt = tstep);

In [78]:
p = [τsE, γE, s0E, τsI, γI, s0I, τrE, kE, IE, wEE, wIE, θE, τrI, kI, II, wEI, wII, θI, τY, βY, γY, ythr, sEthr, kY];
u0 = [0.0, 0.0, 0.0, 0.0, 0.0]

ds = CoupledODEs(model_braint_rhythms, u0, p, diffeq = integ_set);

In [79]:
fp, _, _ = fixedpoints(ds, box, rate_jac);

In [100]:
Jfp = rate_jac(fp[1], p, 0.0)
Jfp_value_vectors = eigen(Jfp)
Jfp_value = Jfp_value_vectors.values
Jfp_vectors = Jfp_value_vectors.vectors;

In [99]:
# 1fp 
Jfp_value_vectors

Eigen{Float64, Float64, SMatrix{5, 5, Float64, 25}, SVector{5, Float64}}
values:
5-element SVector{5, Float64} with indices SOneTo(5):
 -100.0
   -0.9356287690191849
   -0.20498539534386642
   -0.000962691351841854
    0.0010791207936811445
vectors:
5×5 SMatrix{5, 5, Float64, 25} with indices SOneTo(5)×SOneTo(5):
 0.0   0.87436     -0.849873  -0.925154   -0.925298
 0.0   0.00452695  -0.476845  -0.163867   -0.159236
 0.0  -0.485068    -0.139199  -0.337159   -0.339069
 0.0  -0.0135561    0.17595   -0.0596743  -0.0591562
 1.0   0.0          0.0        0.0         0.0

In [97]:
# 2fp 
Jfp_value_vectors

Eigen{ComplexF64, ComplexF64, SMatrix{5, 5, ComplexF64, 25}, SVector{5, ComplexF64}}
values:
5-element SVector{5, ComplexF64} with indices SOneTo(5):
                -100.0 + 0.0im
   -0.9355334264399358 + 0.0im
  -0.20496938191382563 + 0.0im
 1.1369482919028338e-5 - 0.0010195519043432822im
 1.1369482919028338e-5 + 0.0010195519043432822im
vectors:
5×5 SMatrix{5, 5, ComplexF64, 25} with indices SOneTo(5)×SOneTo(5):
 0.0+0.0im    -0.874398+0.0im    -0.8499+0.0im  …   0.925257+0.0im
 0.0+0.0im  -0.00452499+0.0im  -0.476811+0.0im      0.161513-0.00232549im
 0.0+0.0im     0.484999+0.0im  -0.139197+0.0im      0.338054+0.000927702im
 0.0+0.0im    0.0135485+0.0im    0.17591+0.0im     0.0593892-0.000263238im
 1.0+0.0im          0.0+0.0im        0.0+0.0im           0.0+0.0im

In [101]:
# 3fp

Jfp_value_vectors

Eigen{Float64, Float64, SMatrix{5, 5, Float64, 25}, SVector{5, Float64}}
values:
5-element SVector{5, Float64} with indices SOneTo(5):
 -100.0
   -1.0436950357301025
   -0.23004395508578496
    0.014792952969609901
    0.09396802665565543
vectors:
5×5 SMatrix{5, 5, Float64, 25} with indices SOneTo(5)×SOneTo(5):
 0.0   0.827843    -0.820252  -0.883626  -0.882283
 0.0   0.00768177  -0.503829  -0.256643  -0.119694
 0.0  -0.560271    -0.138822  -0.374501  -0.446567
 0.0  -0.0267078    0.232534  -0.114408  -0.0884771
 1.0   0.0          0.0        0.0        0.0